In [13]:
import pandas as pd
from megatron import saving as m
from simple_salesforce import bulk
from connect import td_query as td
from connect import sf_query as sf
from connect import sf_login as sfl
from datetime import date


In [14]:
### This is the BUMBLEBEE output in a dataframe

# connect and query Teradata
bumblebee = td('SELECT DISTINCT account_id,current_owner_id,employee_sf_id FROM sandbox.bumblebee_output where employee_sf_id IS NOT NULL')
bumblebee['account_id'] = bumblebee['account_id'].str.strip()


c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
### This is the current account ownership in SF

# define variables
account_ids = bumblebee['account_id'].tolist()
owner_ids = []

# get data from SOQL query and add to dictionary
for account_id in account_ids:
    soql_output = sf('SELECT Account_ID_18__c,OwnerId FROM Account where Account_ID_18__c',account_id)
    owner_id = soql_output['records'][0]['OwnerId']
    owner_ids.append(owner_id)
    
data = {
    'account_id': account_ids,
    'owner_id': owner_ids
}

current_owner_table = pd.DataFrame(data)


In [16]:
### The filtered table is ready for upload to SF

# This joins the current_owner_table and the bumblebee table
df = bumblebee.join(current_owner_table.set_index('account_id'), on='account_id')

# this filters the table on current_owner_id in Teradata compared to owner_id in SF, and also on the intended employee_sf_id to the current owner_id in SF
filtered_table = df.drop(df[(df['current_owner_id'] != df['owner_id']) & (df['owner_id'] == df['employee_sf_id'])].index)

# this drops unnecesary columns
filtered_table.drop('current_owner_id', axis=1, inplace=True)
filtered_table.drop('owner_id', axis=1, inplace=True)

# columns are renamed to fit Salesforce names
filtered_table.columns = ['Id','OwnerId']


In [17]:
### The data is converted into an ordered dictionary, which is a requirement for SF bulk upload
from collections import OrderedDict

ready_for_upload = []

for row in filtered_table.itertuples():
    d = row._asdict()
    del d['Index']
    ready_for_upload.append(d)


[{'Id': '001C000001UTGRhIAP', 'OwnerId': '005C000000BjmIVIAZ'}, {'Id': '001C000001T3FQrIAN', 'OwnerId': '005C000000BjmIVIAZ'}, {'Id': '001C000001UTGQKIA5', 'OwnerId': '005C000000BjmIVIAZ'}]


In [18]:
### updates are made in Salesforce
from simple_salesforce import Salesforce as sf
    
login_sf = sf(username = 'svc_rao@groupon.com', password = 'Password-1Je8IR8frn0gkbS68Eq4OHZmM', security_token = '')

login_sf.bulk.Account.update(ready_for_upload,batch_size=10000,use_serial=True)


[{'success': True, 'created': False, 'id': '001C000001UTGRhIAP', 'errors': []},
 {'success': True, 'created': False, 'id': '001C000001T3FQrIAN', 'errors': []},
 {'success': True, 'created': False, 'id': '001C000001UTGQKIA5', 'errors': []}]

In [25]:
## upload data to Teradata for metadata reporting
load_date = str(date.today())
filtered_table['load_date'] = load_date

m.df_to_sql(filtered_table,table_name='bumblebee_processed_changes', adapter='teradata')

print("Success")

[10:43:43] [OK] executing "delete from sandbox.bumblebee_processed_changes" succeeded [0:00:00.610211]
[10:43:43] [ALERT] loading data to sandbox.bumblebee_processed_changes table (adapter: teradata) failed! message: [Version 17.20.0.0] [Session 20806900] [Teradata SQL Driver] [Error 528] A failure occurred while executing parameter sets 1 through 3 of a batch request. Details of the failure can be found in the error string that is accessible using the escape functions {fn teradata_nativesql}{fn teradata_get_errors}.
Caused by [Version 17.20.0.0] [Session 20806900] [Teradata Database] [Error 3810] Column/Parameter 'sandbox.bumblebee_processed_changes.id' does not exist.
[10:43:43] [OK] executing "rename table bumblebee_processed_changes to bumblebee_processed_changes_bak_O8XR49" succeeded [0:00:00.362976]
[10:43:43] [ALERT] saving original table as sandbox.bumblebee_processed_changes_bak_O8XR49 and trying to recreate table...
[10:43:44] [OK] executing "
        create multiset table sa